In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from attlayer import AttentionWeightedAverage

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
N_DIMS = 300

In [3]:
from lstm import *

In [4]:
train = pd.read_csv('data\\train.csv').fillna(' ')
test = pd.read_csv('data\\test.csv').fillna(' ')

In [5]:
embed_size = 300
max_features = 394787
maxlen = 200

In [6]:
list_sentences_train = train["comment_text"].values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].values

In [7]:
print("Tokenizing")
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(np.concatenate([list_sentences_train, list_sentences_test])))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

Tokenizing


In [8]:
pretrained = "data\\crawl-300d-2M.vec"
print("Getting", pretrained)
embeddings_index = get_pretrained(pretrained)

Getting data\crawl-300d-2M.vec


In [9]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()

In [10]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector  

In [11]:
net = LstmNet(embed_size, max_features, maxlen, embedding_matrix)

In [12]:
from keras.callbacks import LearningRateScheduler

In [13]:
def halve(epoch):
    lr = 0.002
    return lr/(2**epoch)

In [14]:
lrate = LearningRateScheduler(halve)

In [15]:
net.model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1, callbacks=[lrate])

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:94: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 118436100 elements. This may consume a large amount of memory.
  "This may consume a large amount of memory." % num_elements)


Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 3693s 26ms/step - loss: 0.0501 - acc: 0.9817 - val_loss: 0.0446 - val_acc: 0.9829
Epoch 2/2
143613/143613 [==============================] - 3632s 25ms/step - loss: 0.0318 - acc: 0.9871 - val_loss: 0.0441 - val_acc: 0.9839


In [18]:
y_test = net.predict_proba(X_te)
net.submit()

153164/153164 [==============================] - 103s 671us/step


NameError: name 'list_classes' is not defined

In [22]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
sub = pd.read_csv('data\\sample_submission.csv')
sub[list_classes] = y_test
sub.to_csv('submissions\\lstm9.csv', index=False)

In [20]:
y_test.shape

(153164, 6)